### Importing Packages

In [7]:
import h2o
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Step One is to create a data set, as we saw in earlier videos. If you took my example and modified it, add a comment explaining what your modification was.

In [9]:
h2o.init()
url="http://h2o-public-test-data.S3.amazonaws.com/smalldata/iris/iris_wheader.csv"
iris=h2o.import_file(url)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.261-b12, mixed mode)
  Starting server from C:\Users\Zeus\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Zeus\AppData\Local\Temp\tmpknnx5p5n
  JVM stdout: C:\Users\Zeus\AppData\Local\Temp\tmpknnx5p5n\h2o_Zeus_started_from_python.out
  JVM stderr: C:\Users\Zeus\AppData\Local\Temp\tmpknnx5p5n\h2o_Zeus_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_Zeus_6bk18g
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.757 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


# 2. Step Two is to start h2o, and import your data.

In [10]:
#h2o.init()

# 3. Step Three is to split the data. If you plan to use cross-validation, split into train and test. Otherwise split into train, valid and test.

In [20]:
train,valid,test=iris.split_frame(
    ratios=[0.8,0.1],
    destination_frames=['data_train','data_valid','data_test'],
    seed=123)

train.summary()

,sepal_len,sepal_wid,petal_len,petal_wid,class
type,real,real,real,real,enum
mins,4.3,2.2,1.0,0.1,
mean,5.833884297520661,3.0785123966942143,3.712396694214875,1.1942148760330578,
maxs,7.9,4.4,6.9,2.5,
sigma,0.8223496538383439,0.42273053110507014,1.769960934029048,0.7793477530025944,
zeros,0,0,0,0,
missing,0,0,0,0,0
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


# 4. Step four is to choose either random forest or gbm, and make a model. It can be classification or regression. Then show the results, on both training data and the test data. You can show all the performance stats, or choose just one (e.g. I focused on MAE in the videos).

# I am using GBM

In [21]:
# Importing the sub packages required for GBM

from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [33]:
mGB=H2OGradientBoostingEstimator(model_id='defaults')

y="class"
ignoreFields=[y]
x=[i for i in train.names if i not in ignoreFields]

mGB.train(x,y,train,
         validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [35]:
# mGB.mae(train=True)
# mGB.mae(valid=True)
mGB

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  defaults


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,150.0,25125.0,1.0,5.0,4.526667,2.0,11.0,8.68




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.0006794781525544588
RMSE: 0.02606680173236561
LogLoss: 0.009334691187732336
Mean Per-Class Error: 0.0

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
0,42.0,0.0,0.0,0.0,0 / 42
1,0.0,40.0,0.0,0.0,0 / 40
2,0.0,0.0,39.0,0.0,0 / 39
3,42.0,40.0,39.0,0.0,0 / 121



Top-3 Hit Ratios: 


...

,k,hit_ratio
0,1,0.785714
1,2,1.000000
2,3,1.000000



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
0,,2020-09-03 01:49:02,0.006 sec,0.0,0.666667,1.098612,0.611570,0.666667,1.098612,0.714286
1,,2020-09-03 01:49:02,0.064 sec,1.0,0.600338,0.917377,0.016529,0.619253,0.968112,0.214286
2,,2020-09-03 01:49:02,0.070 sec,2.0,0.539739,0.776588,0.016529,0.578232,0.870424,0.214286
3,,2020-09-03 01:49:02,0.074 sec,3.0,0.484727,0.663961,0.016529,0.545401,0.800140,0.214286
4,,2020-09-03 01:49:02,0.078 sec,4.0,0.435156,0.572157,0.016529,0.518140,0.746393,0.214286
5,,2020-09-03 01:49:02,0.082 sec,5.0,0.390795,0.496312,0.016529,0.495069,0.702128,0.214286
6,,2020-09-03 01:49:02,0.087 sec,6.0,0.351369,0.433040,0.016529,0.477520,0.670595,0.214286
7,,2020-09-03 01:49:02,0.092 sec,7.0,0.316542,0.379773,0.016529,0.461912,0.641192,0.214286
8,,2020-09-03 01:49:02,0.097 sec,8.0,0.285964,0.333792,0.016529,0.439625,0.586959,0.214286
9,,2020-09-03 01:49:02,0.101 sec,9.0,0.258794,0.294504,0.016529,0.426062,0.555749,0.214286



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,petal_wid,225.577362,1.000000,0.599888
1,petal_len,147.651993,0.654551,0.392658
2,sepal_wid,1.517256,0.006726,0.004035
3,sepal_len,1.285605,0.005699,0.003419


In [36]:
p=mRF.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [37]:
p

predict,Iris-setosa,Iris-versicolor,Iris-virginica
Iris-setosa,0.998759,0.000639465,0.0006015
Iris-setosa,0.998213,0.000546027,0.00124052
Iris-setosa,0.998962,0.00058896,0.000448909
Iris-setosa,0.998213,0.000546027,0.00124052
Iris-setosa,0.998989,0.000505272,0.000505506
Iris-setosa,0.998913,0.000589044,0.000498272
Iris-versicolor,0.00062764,0.998518,0.000853968
Iris-versicolor,0.00103436,0.994923,0.00404259
Iris-versicolor,0.000994271,0.994611,0.00439518
Iris-versicolor,0.00071638,0.997344,0.00193937


In [38]:
mRF.model_performance(test)


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.06972500472426629
RMSE: 0.26405492747583087
LogLoss: 0.35507804894370903
Mean Per-Class Error: 0.08333333333333333

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
0,6.0,0.0,0.0,0.000000,0 / 6
1,0.0,5.0,0.0,0.000000,0 / 5
2,0.0,1.0,3.0,0.250000,1 / 4
3,6.0,6.0,3.0,0.066667,1 / 15



Top-3 Hit Ratios: 


,k,hit_ratio
0,1,0.933333
1,2,1.000000
2,3,1.000000


# 5. Step five is then to try some alternative parameters, to build a different model, and show how the results differ.


### Trying to overfit the model

In [39]:
mGB=H2OGradientBoostingEstimator(model_id='overfit',
                                ntrees=10000,
                                max_depth=20)

y="class"
ignoreFields=[y]
x=[i for i in train.names if i not in ignoreFields]

mGB.train(x,y,train,
         validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [40]:
mGB

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  overfit


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,10000.0,30000.0,4381815.0,1.0,7.0,4.381733,2.0,11.0,6.766767




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 3.7070854508678875e-17
RMSE: 6.088583949382556e-09
LogLoss: 1.5204294877302876e-09
Mean Per-Class Error: 0.0

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
0,42.0,0.0,0.0,0.0,0 / 42
1,0.0,40.0,0.0,0.0,0 / 40
2,0.0,0.0,39.0,0.0,0 / 39
3,42.0,40.0,39.0,0.0,0 / 121



Top-3 Hit Ratios: 


...

,k,hit_ratio
0,1,0.857143
1,2,1.000000
2,3,1.000000



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
0,,2020-09-03 01:51:28,0.013 sec,0.0,0.666667,1.098612,0.611570,0.666667,1.098612,0.714286
1,,2020-09-03 01:51:28,0.050 sec,1.0,0.600338,0.917377,0.016529,0.619253,0.968112,0.214286
2,,2020-09-03 01:51:28,0.055 sec,2.0,0.539739,0.776588,0.016529,0.578232,0.870424,0.214286
3,,2020-09-03 01:51:28,0.059 sec,3.0,0.484727,0.663961,0.016529,0.545401,0.800140,0.214286
4,,2020-09-03 01:51:28,0.063 sec,4.0,0.435156,0.572157,0.016529,0.518140,0.746393,0.214286
5,,2020-09-03 01:51:28,0.067 sec,5.0,0.390795,0.496312,0.016529,0.495069,0.702128,0.214286
6,,2020-09-03 01:51:28,0.073 sec,6.0,0.351369,0.433040,0.016529,0.477520,0.670595,0.214286
7,,2020-09-03 01:51:28,0.077 sec,7.0,0.316542,0.379773,0.016529,0.461912,0.641192,0.214286
8,,2020-09-03 01:51:28,0.081 sec,8.0,0.285964,0.333792,0.016529,0.439625,0.586959,0.214286
9,,2020-09-03 01:51:28,0.086 sec,9.0,0.258794,0.294504,0.016529,0.426062,0.555749,0.214286



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,petal_wid,227.639725,1.000000,0.605166
1,petal_len,145.652496,0.639838,0.387208
2,sepal_wid,1.546805,0.006795,0.004112
3,sepal_len,1.321787,0.005806,0.003514


In [41]:
p=mRF.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [42]:
p


predict,Iris-setosa,Iris-versicolor,Iris-virginica
Iris-setosa,0.998759,0.000639465,0.0006015
Iris-setosa,0.998213,0.000546027,0.00124052
Iris-setosa,0.998962,0.00058896,0.000448909
Iris-setosa,0.998213,0.000546027,0.00124052
Iris-setosa,0.998989,0.000505272,0.000505506
Iris-setosa,0.998913,0.000589044,0.000498272
Iris-versicolor,0.00062764,0.998518,0.000853968
Iris-versicolor,0.00103436,0.994923,0.00404259
Iris-versicolor,0.000994271,0.994611,0.00439518
Iris-versicolor,0.00071638,0.997344,0.00193937


In [43]:
mRF.model_performance(test)


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.06972500472426629
RMSE: 0.26405492747583087
LogLoss: 0.35507804894370903
Mean Per-Class Error: 0.08333333333333333

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,Iris-setosa,Iris-versicolor,Iris-virginica,Error,Rate
0,6.0,0.0,0.0,0.000000,0 / 6
1,0.0,5.0,0.0,0.000000,0 / 5
2,0.0,1.0,3.0,0.250000,1 / 4
3,6.0,6.0,3.0,0.066667,1 / 15



Top-3 Hit Ratios: 


,k,hit_ratio
0,1,0.933333
1,2,1.000000
2,3,1.000000


# 6. Step six is to save your script, then close down h2o, and run your script in a fresh session to make sure there are no bugs and the results are repeatable. If your script takes more than a couple of minutes to run, please put the required resources in a comment at the very top. (But first consider if you can get the same results with a smaller data set: this task is about tuning and over-fitting, not about big data.)

### close the H2O connection

In [44]:
h2o.cluster().shutdown()

H2O session _sid_ba75 closed.
